# Cancer Clinical Trial Dataset: The data set was downloaded from https://clinicaltrials.gov/



Aims of project

1. What are the trends in clinical trials over the last decade?
2. How does study duration vary across different cancer types and funding sources?
3. What is the distribution of clinical trials by cancer type and funding source?
4. What are the different types and distribution of active clinical trials?
5. To map clinical trial locations across the United States.


In [ ]:
# Dependencies and Setup
import os

import pandas as pd
import numpy as np
from pathlib import Path
import plotly as py
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import linregress
from scipy.stats import pearsonr
import plotly.graph_objects as go
import plotly.io as pio
from scipy.stats import chi2_contingency

In [7]:
# Read the cancer clinical data 

Cancer_CT_df = pd.read_csv('cancer-ct-data .csv')

In [8]:
Cancer_CT_df

,NCT Number,Study Title,Study URL,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,Enrollment,Funder Type,Study Type,Study Design,Start Date,Primary Completion Date,Completion Date,First Posted,Locations,Study Documents
0,NCT02515227,Safety Study of a Helper Peptide Vaccine Plus ...,https://clinicaltrials.gov/study/NCT02515227,COMPLETED,This study evaluates whether it is safe to adm...,YES,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,Number of Participants With Dose-limiting Toxi...,Number of T Cells in the Tumor Microenvironmen...,...,22,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2016-10-06,2020-03-14,2020-03-14,2015-08-04,"University of Virginia Cancer Center, Charlott...","Study Protocol, Statistical Analysis Plan, and..."
1,NCT03531827,"Combining CRLX101, a Nanoparticle Camptothecin...",https://clinicaltrials.gov/study/NCT03531827,TERMINATED,Background:\n\nSome prostate cancer keeps grow...,YES,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,Percentage of Participants With Anti-tumor Act...,Proportion of Participants With a Sustained >3...,...,4,NIH,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,2019-03-26,2021-05-15,2021-06-01,2018-05-22,"National Institutes of Health Clinical Center,...","Study Protocol and Statistical Analysis Plan, ..."
2,NCT04034927,"Testing the Addition of an Immunotherapy Drug,...",https://clinicaltrials.gov/study/NCT04034927,ACTIVE_NOT_RECRUITING,This phase II trial studies how well olaparib ...,YES,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,"Progression Free Survival (PFS), The count of ...","Objective Response (RECIST 1.1), The objective...",...,61,NIH,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2019-12-04,2021-07-31,2024-09-21,2019-07-29,UC Irvine Health/Chao Family Comprehensive Can...,"Study Protocol and Statistical Analysis Plan, ..."
3,NCT04657094,Acalabrutinib for the Treatment of Relapsed or...,https://clinicaltrials.gov/study/NCT04657094,ACTIVE_NOT_RECRUITING,This phase II trial studies the effect of acal...,YES,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,Autoimmune Hemolytic Anemia (AIHA) - Overall R...,NaN,...,4,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2021-03-16,2023-04-01,2024-12-30,2020-12-08,"City of Hope Medical Center, Duarte, Californi...","Study Protocol and Statistical Analysis Plan, ..."
4,NCT03861091,Prophylactic Risedronate for Patients With Per...,https://clinicaltrials.gov/study/NCT03861091,COMPLETED,This is a double blind randomized controlled s...,YES,Lung Neoplasm,DRUG: risedronate|DRUG: Matching placebo,"Changes in Mean Cortical Thickness, The primar...",4 Time Points Assessing Change in Mean Cortica...,...,84,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2019-07-12,2022-03-02,2022-03-02,2019-03-04,"High Point Regional Medical Center, High Point...","Study Protocol and Statistical Analysis Plan, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,NCT02648477,Pembrolizumab and Doxorubicin Hydrochloride or...,https://clinicaltrials.gov/study/NCT02648477,COMPLETED,This phase II trial studies how well pembroliz...,YES,Estrogen Receptor Negative|Estrogen Receptor P...,DRUG: Anastrozole|DRUG: Doxorubicin Hydrochlor...,"Number of Participants With Overall Response, ...","Clinical Benefit Rate, Per Response Evaluation...",...,30,OTHER,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,2016-03-28,2022-04-01,2023-12-30,2016-01-07,"City of Hope Corona, Corona, California, 92879...","Study Protocol and Statistical Analysis Plan, ..."
712,NCT01931098,Oral Pazopanib Plus Oral Topotecan Metronomic ...,https://clinicaltrials.gov/study/NCT01931098,COMPLETED,Background:\n\nGlioblastoma is the most common...,YES,Glioblast

# Filter data set

In [9]:
Column_to_keep = ["NCT Number", "Study Status", "Conditions", "Interventions", "Sponsor", "Phases", "Funder Type", "Start Date", "Completion Date", "Locations"]
New_Reduced_df = Cancer_CT_df[Column_to_keep]
New_Reduced_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1|PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott..."
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,..."
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...
3,NCT04657094,ACTIVE_NOT_RECRUITING,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,City of Hope Medical Center,PHASE2,OTHER,2021-03-16,2024-12-30,"City of Hope Medical Center, Duarte, Californi..."
4,NCT03861091,COMPLETED,Lung Neoplasm,DRUG: risedronate|DRUG: Matching placebo,Wake Forest University Health Sciences,PHASE2,OTHER,2019-07-12,2022-03-02,"High Point Regional Medical Center, High Point..."
...,...,...,...,...,...,...,...,...,...,...
711,NCT02648477,COMPLETED,Estrogen Receptor Negative|Estrogen Receptor P...,DRUG: Anastrozole|DRUG: Doxorubicin Hydrochlor...,City of Hope Medical Center,PHASE2,OTHER,2016-03-28,2023-12-30,"City of Hope Corona, Corona, California, 92879..."
712,NCT01931098,COMPLETED,Glioblastoma|Glioblastoma Multiforme|Gliosarco...,DRUG: topotecan|DRUG: pazopanib,National Cancer Institute (NCI),PHASE2,NIH,2015-12-10,2019-09-12,"National Institutes of Health Clinical Center,..."
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE1|PHASE2,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77..."
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1|PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo..."


In [11]:
# Create one-hot encoded columns for the phases
phases_dummies = New_Reduced_df['Phases'].str.get_dummies(sep='|')

# Merge the one-hot encoded columns with the original DataFrame
New_Reduced_df = pd.concat([New_Reduced_df, phases_dummies], axis=1)

# Drop the original 'Phases' column if desired
# New_Reduced_df = New_Reduced_df.drop(columns=['Phases'])

# Display the resulting DataFrame
New_Reduced_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,EARLY_PHASE1,PHASE1,PHASE2,PHASE3,PHASE4
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,1,0,0,0
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,0,1,0,0
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",0,0,1,0,0
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,0,0,1,0,0
3,NCT04657094,ACTIVE_NOT_RECRUITING,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,City of Hope Medical Center,PHASE2,OTHER,2021-03-16,2024-12-30,"City of Hope Medical Center, Duarte, Californi...",0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE1,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,1,0,0,0
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE2,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,0,1,0,0
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,1,0,0,0
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0


# Extract State and Zip codes from location column

In [12]:
import pandas as pd

# Function to extract state and zip code
def extract_state_zip(locations):
    if pd.isna(locations) or not isinstance(locations, str):
        return pd.Series([None, None])
    
    # Split locations by '|'
    location_parts = locations.split('|')
    
    # Process each location part and return the last one
    last_location = location_parts[-1].strip()
    
    # Split by commas and extract state and zip code
    parts = last_location.split(',')
    if len(parts) < 4:
        return pd.Series([None, None])
    
    state = parts[-3].strip()
    zip_code = parts[-2].strip()
    
    return pd.Series([state, zip_code])


# Apply the function to the 'Locations' column
New_Reduced_df[['State', 'Zip Code']] = New_Reduced_df['Locations'].apply(extract_state_zip)

New_Reduced_df


,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,EARLY_PHASE1,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,1,0,0,0,Virginia,22908
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,0,1,0,0,Virginia,22908
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",0,0,1,0,0,Maryland,20892
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,0,0,1,0,0,Wisconsin,53226
3,NCT04657094,ACTIVE_NOT_RECRUITING,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,City of Hope Medical Center,PHASE2,OTHER,2021-03-16,2024-12-30,"City of Hope Medical Center, Duarte, Californi...",0,0,1,0,0,California,91010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE1,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,1,0,0,0,Texas,77030
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE2,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,0,1,0,0,Texas,77030
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,1,0,0,0,New York,14263
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0,New York,14263


# Categorizing Cancers based on the keywords

In [13]:
# Keywords for each condition category
lung_keywords = ['Lung', 'Alveolar', 'cigerette', 'smoking' 'Pulmonary', 'Adenocarcinoma']
brain_keywords = ['Brain', 'Glioma', 'Meningioma', 'Neuroblastoma', 'Glioblastoma']
blood_keywords = ['Blood', 'Leukemia', 'Lymphoma', 'Myeloma', 'Anemia', "T-cell"]
prostate_keywords = ['Prostate', 'Prostatic', 'PSA', 'Androgen']
breast_keywords = ['breast', 'Mammary', 'BRCA1', 'BRCA2', 'Luminal', 'HER2']
kidney_keywords = ['Renal', 'RCC', 'Clear Cell', 'Wilms Tumor']
lymphoma_keywords = ['Hodgkin','DLBCL', 'Follicular', 'Burkitt']
pancreatic_keywords = ['Pancreatic', 'Pancreaticobiliary', 'Islet', 'Neuroendocrine']
skin_keywords = ['Melanoma', 'Basal cell', 'Squamous Cell', 'Non-Melanoma']
uterine_keywords = ['Endometrial','Uterine','Myometrial', 'Papillary Serous']
liver_keywords = ['Hepatocellular','Liver','Hepatic', 'Papillary Serous']
# Function to categorize major conditions
def categorize_condition(conditions):
    if any(keyword in conditions for keyword in lung_keywords):
        return 'Lung Cancer'
    elif any(keyword in conditions for keyword in brain_keywords):
        return 'Brain Cancer'
    elif any(keyword in conditions for keyword in blood_keywords):
        return 'Blood Cancer'
    elif any(keyword in conditions for keyword in prostate_keywords):
        return 'Prostate Cancer'
    elif any(keyword in conditions for keyword in breast_keywords):
        return 'breast Cancer'
    elif any(keyword in conditions for keyword in kidney_keywords):
        return 'kidney Cancer'
    elif any(keyword in conditions for keyword in lymphoma_keywords):
        return 'Lymphoma Cancer'
    elif any(keyword in conditions for keyword in uterine_keywords):
        return 'Pancreatic Cancer'
    elif any(keyword in conditions for keyword in breast_keywords):
        return 'Skin Cancer'
    elif any(keyword in conditions for keyword in uterine_keywords):
        return 'Uterine Cancer'
    elif any(keyword in conditions for keyword in liver_keywords):
        return 'liver Cancer'
    else:
        return 'Other'

# Apply the function to the 'Conditions' column
New_Reduced_df['Major Condition'] = New_Reduced_df['Conditions'].apply(categorize_condition)

New_Reduced_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,EARLY_PHASE1,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,1,0,0,0,Virginia,22908,Other
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,0,1,0,0,Virginia,22908,Other
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",0,0,1,0,0,Maryland,20892,Prostate Cancer
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,0,0,1,0,0,Wisconsin,53226,Lung Cancer
3,NCT04657094,ACTIVE_NOT_RECRUITING,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,City of Hope Medical Center,PHASE2,OTHER,2021-03-16,2024-12-30,"City of Hope Medical Center, Duarte, Californi...",0,0,1,0,0,California,91010,Blood Cancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE1,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,1,0,0,0,Texas,77030,Blood Cancer
713,NCT02311998,COMPLETED,B Acute Lymphoblastic Leukemia With t(9;22)(q3...,DRUG: Bosutinib|BIOLOGICAL: Inotuzumab Ozogamicin,M.D. Anderson Cancer Center,PHASE2,OTHER,2015-04-16,2022-03-23,"M D Anderson Cancer Center, Houston, Texas, 77...",0,0,1,0,0,Texas,77030,Blood Cancer
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,1,0,0,0,New York,14263,Other
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0,New York,14263,Other


# Categorizing Cancers based on the keywords

In [14]:
import pandas as pd

# Define keywords for each category
keywords = {
    'Biological': ['biological', 'monoclonal antibody', 'vaccine', 'antibody'],
    'Drug': ['drug', 'chemotherapy', 'medication', 'pill', 'tablet', 'injection'],
    'Procedure': ['procedure', 'surgery', 'operation', 'therapy', 'biopsy', 'collection', 'imaging'],
    'Radiation': ['radiation'],
    'Others': ['laboratory', 'biomarker', 'analysis', 'study', 'questionnaire', 'quality-of-life']
}

def categorize_Interventions(Interventions_text, keywords_dict):
    categories = set()
    
    # Split the text into individual terms
    terms = Interventions_text.split('|')
    
    # Check each term against keywords for each category
    for term in terms:
        term = term.strip().lower()
        for category, key_list in keywords_dict.items():
            if any(keyword.lower() in term for keyword in key_list):
                categories.add(category)
    
    return list(categories) if categories else ['Uncategorized']

# Apply categorization to each intervention and explode the column
New_Reduced_df['Major Interventions'] = New_Reduced_df['Interventions'].apply(lambda x: categorize_Interventions(x, keywords))
New_Reduced_df = New_Reduced_df.explode('Major Interventions')

# Display the DataFrame
New_Reduced_df


,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,EARLY_PHASE1,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,1,0,0,0,Virginia,22908,Other,Drug
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,1,0,0,0,Virginia,22908,Other,Biological
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,0,1,0,0,Virginia,22908,Other,Drug
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",0,0,1,0,0,Virginia,22908,Other,Biological
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",0,0,1,0,0,Maryland,20892,Prostate Cancer,Drug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0,New York,14263,Other,Drug
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0,New York,14263,Other,Others
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",0,0,1,0,0,New York,14263,Other,Biological
715,NCT02242097,ACTIVE_NOT_RECRUITING,Contiguous Stage II Mantle Cell Lymphoma|Nonco...,DRUG: ibrutinib|OTHER: laboratory biomarker an...,Northwestern University,PHASE2,OTHER,2015-01-12,2025-01,"Northwestern University, Chicago, Illinois, 60...",0,0,1,0,0,Utah,84132,Blood Cancer,Drug


# Calculate duration of Clinical Trials

In [15]:

# Convert 'Start Date' and 'Completion Date' to datetime, handling errors
New_Reduced_df['Start Date'] = pd.to_datetime(New_Reduced_df['Start Date'], errors='coerce')
New_Reduced_df['Completion Date'] = pd.to_datetime(New_Reduced_df['Completion Date'], errors='coerce')

# Check for any NaT values in the date columns
print(New_Reduced_df[['Start Date', 'Completion Date']].isna().sum())

# Drop rows with NaT values in 'Start Date' or 'Completion Date'
New_Reduced_df = New_Reduced_df.dropna(subset=['Start Date', 'Completion Date'])

# Calculate the duration of the trial
New_Reduced_df['Duration'] = New_Reduced_df['Completion Date'] - New_Reduced_df['Start Date']

# Calculate the duration in days
New_Reduced_df['Duration (Days)'] = New_Reduced_df['Duration'].dt.days

# Print the DataFrame with the new columns
New_Reduced_df


Start Date         111
Completion Date     98
dtype: int64


/var/folders/_n/lvm0vpcj1rx073qxcpg_05g40000gn/T/ipykernel_5233/1512483141.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Reduced_df['Duration'] = New_Reduced_df['Completion Date'] - New_Reduced_df['Start Date']
/var/folders/_n/lvm0vpcj1rx073qxcpg_05g40000gn/T/ipykernel_5233/1512483141.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  New_Reduced_df['Duration (Days)'] = New_Reduced_df['Duration'].dt.days


,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,...,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions,Duration,Duration (Days)
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,1,0,0,0,Virginia,22908,Other,Drug,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,1,0,0,0,Virginia,22908,Other,Biological,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,0,1,0,0,Virginia,22908,Other,Drug,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,0,1,0,0,Virginia,22908,Other,Biological,1255 days,1255
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",...,0,1,0,0,Maryland,20892,Prostate Cancer,Drug,798 days,798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,1,0,0,0,New York,14263,Other,Others,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,1,0,0,0,New York,14263,Other,Biological,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,0,1,0,0,New York,14263,Other,Drug,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,0,1,0,0,New York,14263,Other,Others,2141 days,2141


# Plot interactive box plot: Major Condition, Funding type, and duration of clinical trials

In [48]:
# Plot using Plotly Express with Phases as color
fig = px.box(New_Reduced_df, x='Major Condition', y='Duration (Days)', color='Funder Type',
             title='Study Duration by Condition and Funder Type',
             labels={'Duration (Days)': 'Duration (Days)', 'Major Condition': 'Major Condition', 'Funder Type': 'Funder Type'})

# Customize layout: Set background to white and add interactive buttons for Funder Type
fig.update_layout(
    plot_bgcolor='white',  # Set the background color to white
    xaxis=dict(showgrid=True, gridcolor='LightGrey'),
    yaxis=dict(showgrid=True, gridcolor='LightGrey'),
    updatemenus=[
        dict(
            type='dropdown',
            buttons=[
                dict(
                    args=[{'visible': [True, False, False, False]}],
                    label="Network",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, True, False, False]}],
                    label="Industry",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, False, True, False]}],
                    label="NIH",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, False, False, True]}],
                    label="Other",
                    method="update"
                ),
                dict(
                    args=[{'visible': [True, True, True, True]}],
                    label="All",
                    method="update"
                )
            ],
            direction='down',  # This creates a dropdown menu
            showactive=True
        )
    ]
)

# Show the plot
fig.show()

# Save the plot as an image
pio.write_image(fig, 'Condition_Funder.png', format='png')

# Plot interactive box plot: Major Condition and Funding type

In [44]:
# Aggregate data for the bar plot (e.g., counts of studies)
agg_df = New_Reduced_df.groupby(['Major Condition', 'Funder Type']).size().reset_index(name='Counts')

# Create a bar plot using Plotly Express
fig = px.bar(agg_df, x='Major Condition', y='Counts', color='Funder Type',
             title='Number of Studies by Major Condition and Funder Type',
             labels={'Counts': 'Number of Studies', 'Major Condition': 'Major Condition', 'Funder Type': 'Funder Type'},
             barmode='group')  # Use 'group' to show bars side by side

# Customize layout: Set background to white and add interactive buttons for Funder Type
fig.update_layout(
    plot_bgcolor='white',  # Set the background color to white
    xaxis=dict(showgrid=True, gridcolor='LightGrey'),
    yaxis=dict(showgrid=True, gridcolor='LightGrey'),
    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{'visible': [True, False, False, False]}],
                    label="Network",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, True, False, False]}],
                    label="Industry",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, False, True, False]}],
                    label="NIH",
                    method="update"
                ),
                dict(
                    args=[{'visible': [False, False, False, True]}],
                    label="Other",
                    method="update"
                ),
                dict(
                    args=[{'visible': [True, True, True, True]}],
                    label="All",
                    method="update"
                )
            ],
            direction="down",
            showactive=True,
            x=1.15,  # Adjust the x-position
            xanchor='right',
            y=1.15,  # Adjust the y-position
            yanchor='top'
        ),
    ]
)

# Show the plot
fig.show()

# Save the figure as a PNG image
pio.write_image(fig, 'Condition_Funder_Bar.png', format='png')

# Plot interactive pie plot: Major Condition and Funding type

In [43]:
# Aggregate data for pie chart (e.g., counts of studies by Major Condition)
agg_df = New_Reduced_df.groupby(['Major Condition', 'Funder Type']).size().reset_index(name='Counts')

# Create pie chart using Plotly Express
fig = px.pie(agg_df, names='Major Condition', values='Counts', color='Funder Type',
             title='Distribution of Studies by Major Condition and Funder Type',
             labels={'Counts': 'Number of Studies', 'Major Condition': 'Major Condition', 'Funder Type': 'Funder Type'})

# Customize layout and add interactive buttons for Funder Type
fig.update_layout(
    plot_bgcolor='white',  # Set the background color to white
    legend_title_text='Funder Type',
    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{'labels': [agg_df[agg_df['Funder Type'] == funder]['Major Condition'].tolist() for funder in agg_df['Funder Type'].unique()],
                           'values': [agg_df[agg_df['Funder Type'] == funder]['Counts'].tolist() for funder in agg_df['Funder Type'].unique()]}],
                    label="All",
                    method="update"
                ),
                dict(
                    args=[{'labels': [agg_df[agg_df['Funder Type'] == 'Government']['Major Condition'].tolist()],
                           'values': [agg_df[agg_df['Funder Type'] == 'Government']['Counts'].tolist()]}],
                    label="Government",
                    method="update"
                ),
                dict(
                    args=[{'labels': [agg_df[agg_df['Funder Type'] == 'Industry']['Major Condition'].tolist()],
                           'values': [agg_df[agg_df['Funder Type'] == 'Industry']['Counts'].tolist()]}],
                    label="Industry",
                    method="update"
                ),
                dict(
                    args=[{'labels': [agg_df[agg_df['Funder Type'] == 'Other']['Major Condition'].tolist()],
                           'values': [agg_df[agg_df['Funder Type'] == 'Other']['Counts'].tolist()]}],
                    label="Other",
                    method="update"
                )
            ],
            direction="down",
            showactive=True,
            x=1.15,  # Adjust the x-position
            xanchor='right',
            y=1.15,  # Adjust the y-position
            yanchor='top'
        ),
    ]
)

# Show the plot
fig.show()

# Save the figure as a PNG image
pio.write_image(fig, 'Condition_Funder_Pie.png', format='png')

# Plot interactive box plot: Major Condition, Major interventions, and trial duration

In [19]:
# Convert Duration (Days) to numeric if it's not already
New_Reduced_df['Duration (Days)'] = pd.to_numeric(New_Reduced_df['Duration (Days)'], errors='coerce')

# Plot using Plotly Express with Phases as color
fig = px.box(New_Reduced_df, x='Major Condition', y='Duration (Days)', color='Major Interventions',
             title='Study Duration by Condition and Phases',
             labels={'Duration (Days)': 'Duration (Days)'})

# Show the plot
fig.show()

/var/folders/_n/lvm0vpcj1rx073qxcpg_05g40000gn/T/ipykernel_5233/2411840325.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Create Data Frame for active_not_recruiting clinical trials

In [20]:
# Filter for active studies 
active_studies_df = New_Reduced_df[New_Reduced_df['Study Status'] == 'ACTIVE_NOT_RECRUITING']
active_studies_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,...,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions,Duration,Duration (Days)
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,...,0,1,0,0,Wisconsin,53226,Lung Cancer,Procedure,1753 days,1753
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,...,0,1,0,0,Wisconsin,53226,Lung Cancer,Drug,1753 days,1753
2,NCT04034927,ACTIVE_NOT_RECRUITING,Fallopian Tube Endometrioid Tumor|Fallopian Tu...,PROCEDURE: Biospecimen Collection|PROCEDURE: C...,National Cancer Institute (NCI),PHASE2,NIH,2019-12-04,2024-09-21,UC Irvine Health/Chao Family Comprehensive Can...,...,0,1,0,0,Wisconsin,53226,Lung Cancer,Biological,1753 days,1753
3,NCT04657094,ACTIVE_NOT_RECRUITING,Autoimmune Hemolytic Anemia|Chronic Lymphocyti...,DRUG: Acalabrutinib,City of Hope Medical Center,PHASE2,OTHER,2021-03-16,2024-12-30,"City of Hope Medical Center, Duarte, Californi...",...,0,1,0,0,California,91010,Blood Cancer,Drug,1385 days,1385
7,NCT03737994,ACTIVE_NOT_RECRUITING,Lung Non-Squamous Non-Small Cell Carcinoma|Sta...,DRUG: Alectinib|DRUG: Brigatinib|DRUG: Carbopl...,National Cancer Institute (NCI),PHASE2,NIH,2019-07-25,2024-09-21,"CTCA at Western Regional Medical Center, Goody...",...,0,1,0,0,Wyoming,82801,Lung Cancer,Drug,1885 days,1885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,NCT03873805,ACTIVE_NOT_RECRUITING,Castration-Resistant Prostate Carcinoma|Metast...,BIOLOGICAL: Autologous Anti-PSCA-CAR-4-1BB/TCR...,City of Hope Medical Center,PHASE1,OTHER,2019-08-20,2024-12-31,"City of Hope Medical Center, Duarte, Californi...",...,1,0,0,0,California,91010,Prostate Cancer,Drug,1960 days,1960
704,NCT03873805,ACTIVE_NOT_RECRUITING,Castration-Resistant Prostate Carcinoma|Metast...,BIOLOGICAL: Autologous Anti-PSCA-CAR-4-1BB/TCR...,City of Hope Medical Center,PHASE1,OTHER,2019-08-20,2024-12-31,"City of Hope Medical Center, Duarte, Californi...",...,1,0,0,0,California,91010,Prostate Cancer,Biological,1960 days,1960
706,NCT02443077,ACTIVE_NOT_RECRUITING,Recurrent Diffuse Large B-Cell Lymphoma Activa...,PROCEDURE: Autologous Bone Marrow Transplantat...,National Cancer Institute (NCI),PHASE3,NIH,2016-10-12,2025-05-10,"Anchorage Associates in Radiation Medicine, An...",...,0,0,1,0,Riyadh,11211,Blood Cancer,Procedure,3132 days,3132
706,NCT02443077,ACTIVE_NOT_RECRUITING,Recurrent Diffuse Large B-Cell Lymphoma Activa...,PROCEDURE: Autologous Bone Marrow Transplantat...,National Cancer Institute (NCI),PHASE3,NIH,2016-10-12,2025-05-10,"Anchorage Associates in Radiation Medicine, An...",...,0,0,1,0,Riyadh,11211,Blood Cancer,Others,3132 days,3132


# Create Data Frame for recruiting clinical trials

In [21]:
# Filter for actively recruiting trials  
recruiting_studies_df = New_Reduced_df[New_Reduced_df['Study Status'] == 'RECRUITING']
recruiting_studies_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,...,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions,Duration,Duration (Days)
64,NCT03155620,RECRUITING,Advanced Malignant Solid Neoplasm|Ann Arbor St...,PROCEDURE: Biopsy|PROCEDURE: Biospecimen Colle...,National Cancer Institute (NCI),PHASE2,NIH,2017-07-31,2027-09-30,"Children's Hospital of Alabama, Birmingham, Al...",...,0,1,0,0,San Juan,00926,Brain Cancer,Procedure,3713 days,3713
64,NCT03155620,RECRUITING,Advanced Malignant Solid Neoplasm|Ann Arbor St...,PROCEDURE: Biopsy|PROCEDURE: Biospecimen Colle...,National Cancer Institute (NCI),PHASE2,NIH,2017-07-31,2027-09-30,"Children's Hospital of Alabama, Birmingham, Al...",...,0,1,0,0,San Juan,00926,Brain Cancer,Others,3713 days,3713
64,NCT03155620,RECRUITING,Advanced Malignant Solid Neoplasm|Ann Arbor St...,PROCEDURE: Biopsy|PROCEDURE: Biospecimen Colle...,National Cancer Institute (NCI),PHASE2,NIH,2017-07-31,2027-09-30,"Children's Hospital of Alabama, Birmingham, Al...",...,0,1,0,0,San Juan,00926,Brain Cancer,Drug,3713 days,3713
166,NCT04203316,RECRUITING,Recurrent Acute Myeloid Leukemia|Refractory Ac...,PROCEDURE: Biospecimen Collection|PROCEDURE: B...,Children's Oncology Group,PHASE2,NETWORK,2023-08-14,2030-12-31,"Children's Hospital of Alabama, Birmingham, Al...",...,0,1,0,0,Quebec,H3T 1C5,Blood Cancer,Procedure,2696 days,2696
166,NCT04203316,RECRUITING,Recurrent Acute Myeloid Leukemia|Refractory Ac...,PROCEDURE: Biospecimen Collection|PROCEDURE: B...,Children's Oncology Group,PHASE2,NETWORK,2023-08-14,2030-12-31,"Children's Hospital of Alabama, Birmingham, Al...",...,0,1,0,0,Quebec,H3T 1C5,Blood Cancer,Drug,2696 days,2696
178,NCT04343157,RECRUITING,"Brain Metastases, Adult|Neurocognitive Functio...",RADIATION: Cognitive Sparing Brain Stereotacti...,Jona Hattangadi-Gluth,PHASE2,OTHER,2019-05-01,2024-12-31,"Moores Cancer Center, San Diego, California, 9...",...,0,1,0,0,California,92037,Brain Cancer,Procedure,2071 days,2071
178,NCT04343157,RECRUITING,"Brain Metastases, Adult|Neurocognitive Functio...",RADIATION: Cognitive Sparing Brain Stereotacti...,Jona Hattangadi-Gluth,PHASE2,OTHER,2019-05-01,2024-12-31,"Moores Cancer Center, San Diego, California, 9...",...,0,1,0,0,California,92037,Brain Cancer,Radiation,2071 days,2071
414,NCT04393350,RECRUITING,Kidney Cancer|Stage III Renal Cell Cancer AJCC...,DRUG: Lenvatinib|DRUG: Lenvatinib Mesylate|BIO...,Emory University,PHASE2,OTHER,2020-06-22,2024-08-05,Emory University Hospital/Winship Cancer Insti...,...,0,1,0,0,Georgia,30322,kidney Cancer,Drug,1505 days,1505
414,NCT04393350,RECRUITING,Kidney Cancer|Stage III Renal Cell Cancer AJCC...,DRUG: Lenvatinib|DRUG: Lenvatinib Mesylate|BIO...,Emory University,PHASE2,OTHER,2020-06-22,2024-08-05,Emory University Hospital/Winship Cancer Insti...,...,0,1,0,0,Georgia,30322,kidney Cancer,Others,1505 days,1505
414,NCT04393350,RECRUITING,Kidney Cancer|Stage III Renal Cell Cancer AJCC...,DRUG: Lenvatinib|DRUG: Lenvatinib Mesylate|BIO...,Emory University,PHASE2,OTHER,2020-06-22,2024-08-05,Emory University Hospital/Winship Cancer Insti...,...,0,1,0,0,Georgia,30322,kidney Cancer,Biological,1505 days,1505


# # Create Data Frame for completed clinical trials

In [49]:
# Filter for completed studies 
completed_studies_df = New_Reduced_df[New_Reduced_df['Study Status'] == 'COMPLETED']
completed_studies_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,...,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions,Duration,Duration (Days)
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,1,0,0,0,Virginia,22908,Other,Drug,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE1,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,1,0,0,0,Virginia,22908,Other,Biological,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,0,1,0,0,Virginia,22908,Other,Drug,1255 days,1255
0,NCT02515227,COMPLETED,Melanoma,BIOLOGICAL: 6MHP|DRUG: Pembrolizumab,"Craig L Slingluff, Jr",PHASE2,OTHER,2016-10-06,2020-03-14,"University of Virginia Cancer Center, Charlott...",...,0,1,0,0,Virginia,22908,Other,Biological,1255 days,1255
4,NCT03861091,COMPLETED,Lung Neoplasm,DRUG: risedronate|DRUG: Matching placebo,Wake Forest University Health Sciences,PHASE2,OTHER,2019-07-12,2022-03-02,"High Point Regional Medical Center, High Point...",...,0,1,0,0,North Carolina,27157,Lung Cancer,Drug,964 days,964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,1,0,0,0,New York,14263,Other,Others,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE1,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,1,0,0,0,New York,14263,Other,Biological,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,0,1,0,0,New York,14263,Other,Drug,2141 days,2141
714,NCT02166905,COMPLETED,Fallopian Tube Carcinoma|Ovarian Carcinoma|Pri...,BIOLOGICAL: DEC-205/NY-ESO-1 Fusion Protein CD...,Roswell Park Cancer Institute,PHASE2,OTHER,2014-10-10,2020-08-20,"Roswell Park Cancer Institute, Buffalo, New Yo...",...,0,1,0,0,New York,14263,Other,Others,2141 days,2141


# Create Data Frame for terminated clinical trials

In [23]:
# Filter for active studies 
terminated_studies_df = New_Reduced_df[New_Reduced_df['Study Status'] == 'TERMINATED']
terminated_studies_df

,NCT Number,Study Status,Conditions,Interventions,Sponsor,Phases,Funder Type,Start Date,Completion Date,Locations,...,PHASE1,PHASE2,PHASE3,PHASE4,State,Zip Code,Major Condition,Major Interventions,Duration,Duration (Days)
1,NCT03531827,TERMINATED,Metastatic Castration Resistant Prostate Cance...,DRUG: enzalutamide|DRUG: CRLX101,National Cancer Institute (NCI),PHASE2,NIH,2019-03-26,2021-06-01,"National Institutes of Health Clinical Center,...",...,0,1,0,0,Maryland,20892,Prostate Cancer,Drug,798 days,798
9,NCT03190967,TERMINATED,Breast Cancer|Brain Metastasis|Brain Cancer,DRUG: T-DM1|DRUG: TMZ,National Cancer Institute (NCI),PHASE1,NIH,2018-04-18,2023-06-13,"National Institutes of Health Clinical Center,...",...,1,0,0,0,Maryland,20892,Brain Cancer,Drug,1882 days,1882
9,NCT03190967,TERMINATED,Breast Cancer|Brain Metastasis|Brain Cancer,DRUG: T-DM1|DRUG: TMZ,National Cancer Institute (NCI),PHASE2,NIH,2018-04-18,2023-06-13,"National Institutes of Health Clinical Center,...",...,0,1,0,0,Maryland,20892,Brain Cancer,Drug,1882 days,1882
11,NCT02565901,TERMINATED,Castration-Resistant Prostate Carcinoma|Metast...,DRUG: Carboplatin|DRUG: Docetaxel|OTHER: Labor...,University of Washington,PHASE1,OTHER,2016-02-29,2020-06-16,Fred Hutch/University of Washington Cancer Con...,...,1,0,0,0,Washington,98109,Prostate Cancer,Drug,1569 days,1569
11,NCT02565901,TERMINATED,Castration-Resistant Prostate Carcinoma|Metast...,DRUG: Carboplatin|DRUG: Docetaxel|OTHER: Labor...,University of Washington,PHASE1,OTHER,2016-02-29,2020-06-16,Fred Hutch/University of Washington Cancer Con...,...,1,0,0,0,Washington,98109,Prostate Cancer,Others,1569 days,1569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,NCT04088188,TERMINATED,Advanced Cholangiocarcinoma|Metastatic Cholang...,DRUG: Cisplatin|DRUG: Gemcitabine|DRUG: Ivosid...,Academic and Community Cancer Research United,PHASE1,OTHER,2021-01-25,2023-12-01,"Mayo Clinic in Arizona, Scottsdale, Arizona, 8...",...,1,0,0,0,Wisconsin,53226,Other,Drug,1040 days,1040
681,NCT02332928,TERMINATED,Breast Cancer - Female,DRUG: Melatonin|DRUG: Placebo,Virginia Commonwealth University,PHASE3,OTHER,2015-03-25,2021-06-02,Virginia Commonwealth University/Massey Cancer...,...,0,0,1,0,Virginia,23298,Other,Drug,2261 days,2261
683,NCT02899728,TERMINATED,Extensive Stage Lung Small Cell Carcinoma,DRUG: Carboplatin|DRUG: Cediranib|DRUG: Cedira...,National Cancer Institute (NCI),PHASE2,NIH,2018-03-30,2020-03-17,"Los Angeles County-USC Medical Center, Los Ang...",...,0,1,0,0,Virginia,23298,Lung Cancer,Drug,718 days,718
702,NCT04616547,TERMINATED,Castration-Resistant Prostate Carcinoma|Metast...,OTHER: Questionnaire Administration|RADIATION:...,National Cancer Institute (NCI),PHASE2,NIH,2021-12-18,2022-05-11,"University of Kentucky/Markey Cancer Center, L...",...,0,1,0,0,Kentucky,40536,Lung Cancer,Radiation,144 days,144


# Create Data Frame for active clinical trial location 

In [26]:
selected_columns = ['NCT Number', 'Major Condition','Major Interventions', 'Sponsor', 'Phases', 'State', 'Zip Code']
active_study_location_df = active_studies_df[selected_columns]
# Save to CSV
active_study_location_df.to_csv('active_location.csv', index=False)
active_study_location_df.to_json('activelocation.json', orient='records')
active_study_location_df

,NCT Number,Major Condition,Major Interventions,Sponsor,Phases,State,Zip Code
2,NCT04034927,Lung Cancer,Procedure,National Cancer Institute (NCI),PHASE2,Wisconsin,53226
2,NCT04034927,Lung Cancer,Drug,National Cancer Institute (NCI),PHASE2,Wisconsin,53226
2,NCT04034927,Lung Cancer,Biological,National Cancer Institute (NCI),PHASE2,Wisconsin,53226
3,NCT04657094,Blood Cancer,Drug,City of Hope Medical Center,PHASE2,California,91010
7,NCT03737994,Lung Cancer,Drug,National Cancer Institute (NCI),PHASE2,Wyoming,82801
...,...,...,...,...,...,...,...
704,NCT03873805,Prostate Cancer,Drug,City of Hope Medical Center,PHASE1,California,91010
704,NCT03873805,Prostate Cancer,Biological,City of Hope Medical Center,PHASE1,California,91010
706,NCT02443077,Blood Cancer,Procedure,National Cancer Institute (NCI),PHASE3,Riyadh,11211
706,NCT02443077,Blood Cancer,Others,National Cancer Institute (NCI),PHASE3,Riyadh,11211


# Create Data Frame for actively recruiting clinical trial location 

In [27]:
selected_columns = ['NCT Number', 'Major Condition','Major Interventions', 'Sponsor', 'Phases', 'State', 'Zip Code']
recruiting_study_location_df = recruiting_studies_df[selected_columns]
# Save to CSV
recruiting_study_location_df.to_csv('recruiting_location.csv', index=False)
recruiting_study_location_df.to_json('recruitinglocation.json', orient='records')
recruiting_study_location_df

,NCT Number,Major Condition,Major Interventions,Sponsor,Phases,State,Zip Code
64,NCT03155620,Brain Cancer,Procedure,National Cancer Institute (NCI),PHASE2,San Juan,00926
64,NCT03155620,Brain Cancer,Others,National Cancer Institute (NCI),PHASE2,San Juan,00926
64,NCT03155620,Brain Cancer,Drug,National Cancer Institute (NCI),PHASE2,San Juan,00926
166,NCT04203316,Blood Cancer,Procedure,Children's Oncology Group,PHASE2,Quebec,H3T 1C5
166,NCT04203316,Blood Cancer,Drug,Children's Oncology Group,PHASE2,Quebec,H3T 1C5
178,NCT04343157,Brain Cancer,Procedure,Jona Hattangadi-Gluth,PHASE2,California,92037
178,NCT04343157,Brain Cancer,Radiation,Jona Hattangadi-Gluth,PHASE2,California,92037
414,NCT04393350,kidney Cancer,Drug,Emory University,PHASE2,Georgia,30322
414,NCT04393350,kidney Cancer,Others,Emory University,PHASE2,Georgia,30322
414,NCT04393350,kidney Cancer,Biological,Emory University,PHASE2,Georgia,30322


# Create Data Frame for completed clinical trial location 

In [31]:
selected_columns = ['NCT Number', 'Major Condition','Major Interventions', 'Sponsor', 'Phases', 'State', 'Zip Code']
completed_study_location_df = completed_studies_df[selected_columns]
# Save to CSV
completed_study_location_df.to_csv('completed_location.csv', index=False)
completed_study_location_df.to_json('completedlocation.json', orient='records')
completed_study_location_df

,NCT Number,Major Condition,Major Interventions,Sponsor,Phases,State,Zip Code
0,NCT02515227,Other,Drug,"Craig L Slingluff, Jr",PHASE1,Virginia,22908
0,NCT02515227,Other,Biological,"Craig L Slingluff, Jr",PHASE1,Virginia,22908
0,NCT02515227,Other,Drug,"Craig L Slingluff, Jr",PHASE2,Virginia,22908
0,NCT02515227,Other,Biological,"Craig L Slingluff, Jr",PHASE2,Virginia,22908
4,NCT03861091,Lung Cancer,Drug,Wake Forest University Health Sciences,PHASE2,North Carolina,27157
...,...,...,...,...,...,...,...
714,NCT02166905,Other,Others,Roswell Park Cancer Institute,PHASE1,New York,14263
714,NCT02166905,Other,Biological,Roswell Park Cancer Institute,PHASE1,New York,14263
714,NCT02166905,Other,Drug,Roswell Park Cancer Institute,PHASE2,New York,14263
714,NCT02166905,Other,Others,Roswell Park Cancer Institute,PHASE2,New York,14263


# # Create Data Frame for terminated clinical trial location 

In [34]:
selected_columns = ['NCT Number', 'Major Condition','Major Interventions', 'Sponsor', 'Phases', 'State', 'Zip Code']
terminated_study_location_df = terminated_studies_df[selected_columns]
# Save to CSV
terminated_study_location_df.to_csv('terminated_location.csv', index=False)
terminated_study_location_df.to_json('terminatedlocation.json', orient='records')
terminated_study_location_df

,NCT Number,Major Condition,Major Interventions,Sponsor,Phases,State,Zip Code
1,NCT03531827,Prostate Cancer,Drug,National Cancer Institute (NCI),PHASE2,Maryland,20892
9,NCT03190967,Brain Cancer,Drug,National Cancer Institute (NCI),PHASE1,Maryland,20892
9,NCT03190967,Brain Cancer,Drug,National Cancer Institute (NCI),PHASE2,Maryland,20892
11,NCT02565901,Prostate Cancer,Drug,University of Washington,PHASE1,Washington,98109
11,NCT02565901,Prostate Cancer,Others,University of Washington,PHASE1,Washington,98109
...,...,...,...,...,...,...,...
678,NCT04088188,Other,Drug,Academic and Community Cancer Research United,PHASE1,Wisconsin,53226
681,NCT02332928,Other,Drug,Virginia Commonwealth University,PHASE3,Virginia,23298
683,NCT02899728,Lung Cancer,Drug,National Cancer Institute (NCI),PHASE2,Virginia,23298
702,NCT04616547,Lung Cancer,Radiation,National Cancer Institute (NCI),PHASE2,Kentucky,40536


# Trends in Clinical Trials Over the Last Decade


In [36]:

# Function to create smoothed line traces for each study status
def create_trace(trial_status, color, window=3):
    filtered_df = New_Reduced_df[New_Reduced_df['Study Status'] == trial_status]
    trial_counts = filtered_df.groupby(filtered_df['Start Date'].dt.to_period('M'))['NCT Number'].nunique().reset_index()
    trial_counts.columns = ['Start Date', 'Number of Trials']
    trial_counts['Start Date'] = trial_counts['Start Date'].dt.to_timestamp()
    trial_counts['Smoothed Trials'] = trial_counts['Number of Trials'].rolling(window=window, center=True).mean()
    return go.Scatter(
        x = trial_counts['Start Date'],
        y = trial_counts['Smoothed Trials'],
        mode = 'lines',
        name = trial_status,
        marker = dict(color = color)
    )

# Create traces for each study status with smoothing window of 6 months
trace_recruiting = create_trace('RECRUITING', 'aquamarine', window=6)
trace_active = create_trace('ACTIVE_NOT_RECRUITING', 'coral', window=6)
trace_completed = create_trace('COMPLETED', 'purple', window=6)
trace_terminated = create_trace('TERMINATED', 'olive', window=6)

# Create the figure and add the traces
fig = go.Figure()

fig.add_trace(trace_recruiting)
fig.add_trace(trace_active)
fig.add_trace(trace_completed)
fig.add_trace(trace_terminated)

# Customize layout
fig.update_layout(
    title='Number of Clinical Trials by Study Status Over Time (Smoothed)',
    xaxis_title='Date',
    yaxis_title='Number of Trials (Smoothed)',
    template='plotly_white'
)

# Show the plot
fig.show()

pio.write_image(fig, 'clinical_trials_decade_static.png', format='png')



/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/opt/anaconda3/lib/python3.1

# Plot interactive bar chart: Major conditions, and study status

In [37]:
def create_trace(trial_status, color):
    filtered_df = New_Reduced_df[New_Reduced_df['Study Status'] == trial_status]
    trial_counts = filtered_df.groupby('Major Condition')['NCT Number'].nunique().reset_index()
    trial_counts.columns = ['Major Condition', 'Number of Trials']
    return go.Bar(
        x = trial_counts['Major Condition'],
        y = trial_counts['Number of Trials'],
        name = trial_status,
        marker = dict(color = color)
    )

# Create traces for each study status
trace_recruiting = create_trace('RECRUITING', 'aquamarine')
trace_active = create_trace('ACTIVE_NOT_RECRUITING', 'coral')
trace_completed = create_trace('COMPLETED', 'purple')
trace_terminated = create_trace('TERMINATED', 'olive')

# Create the figure and add the traces
fig = go.Figure()

fig.add_trace(trace_recruiting)
fig.add_trace(trace_active)
fig.add_trace(trace_completed)
fig.add_trace(trace_terminated)

# Customize layout
fig.update_layout(
    title='Number of Clinical Trials by Target Organ and Study Status',
    xaxis_title='Target Organ',
    yaxis_title='Number of Trials',
    template='plotly_white',
    barmode='group',  
    updatemenus=[{
        'buttons': [
            {
                'label': 'All',
                'method': 'update',
                'args': [{'visible': [True, True, True, True]}]  # Show all traces
            },
            {
                'label': 'Recruiting',
                'method': 'update',
                'args': [{'visible': [True, False, False, False]}]  # Show Recruiting traces
            },
            {
                'label': 'Active',
                'method': 'update',
                'args': [{'visible': [False, True, False, False]}]  # Show only Active trace
            },
            {
                'label': 'Completed',
                'method': 'update',
                'args': [{'visible': [False, False, True, False]}]  # Show only Completed trace
            },
            {
                'label': 'Terminated',
                'method': 'update',
                'args': [{'visible': [False, False, False, True]}]  # Show only Terminated trace
            }
        ],
        'direction': 'down',
        'showactive': True
    }]
)
# Show the plot
fig.show()

# Save the figure as a PNG image
pio.write_image(fig, 'clinical_trials_static.png', format='png')

# Plot interactive bar chart: Major conditions, and phases

In [50]:
# Group by 'Major Condition' and 'Phases' and count the occurrences
phases_counts = active_studies_df.groupby(['Major Condition', 'Phases']).size().reset_index(name='Counts')

# Define a list of colors for different phases
colors = ['#5DADE2', '#E07B39', '#808000', '#BC8F8F','#9467bd','#8c564b','#e377c2','#7f7f7f', '#bcbd22','#17becf']

# Create a function to create traces for each phase
def create_trace(phase, color):
    phase_data = phases_counts[phases_counts['Phases'] == phase]
    return go.Bar(
        x=phase_data['Major Condition'],
        y=phase_data['Counts'],
        name=phase,
        marker=dict(color=color)
    )

# Create a function to update the plot based on the button click
def update_plot(button):
    # Create a bar plot
    fig = go.Figure()

    # Add a trace for each unique Phase, with a unique color
    for i, phase in enumerate(phases_counts['Phases'].unique()):
        fig.add_trace(create_trace(phase, colors[i % len(colors)]))

    # Customize layout
    fig.update_layout(
        title='Number of Active Studies by Major Condition and Phases',
        xaxis_title='Major Condition',
        yaxis_title='Number of Active Studies',
        template='plotly_white',
        barmode='stack',  # Stack bars for different phases
        updatemenus=[{
            'buttons': [
                {
                    'label': 'All',
                    'method': 'update',
                    'args': [{'visible': [True] * len(phases_counts['Phases'].unique())}]  # Show all traces
                },
                *[
                    {
                        'label': phase,
                        'method': 'update',
                        'args': [{'visible': [phase == p for p in phases_counts['Phases'].unique()]}]  # Show only selected phase
                    }
                    for phase in phases_counts['Phases'].unique()
                ]
            ],
            'direction': 'down',
            'showactive': True
        }]
    )

    # Show the plot
    fig.show()

    # Save the figure as a PNG image
    pio.write_image(fig, 'Condition_phase.png', format='png')

# Create a button widget
button = widgets.Button(description="Update Plot")

# Link the button to the update_plot function
button.on_click(update_plot)

# Display the button
display(button)

# Optionally, you can call the function directly to display the initial plot
update_plot(None)

Button(description='Update Plot', style=ButtonStyle())

In [62]:
pip install psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 2.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [71]:
username = os.getenv('DB_USERNAME', 'default_username')
password = os.getenv('DB_PASSWORD', 'default_password')
host = os.getenv('DB_HOST', 'localhost')
database = os.getenv('DB_NAME', 'default_database')
port = '5432'

if None in [username, password, host, database]:
    print("Some required environment variables are missing.")
    username = input("Enter DB_USERNAME: ")
    password = input("Enter DB_PASSWORD: ")
    host = input("Enter DB_HOST: ")
    database = input("Enter DB_NAME: ")


In [73]:

from sqlalchemy import create_engine


# Get database credentials from environment variables
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME')
port = '5432'  # Default PostgreSQL port

# Check if any environment variables are missing
if None in [username, password, host, database]:
    raise ValueError("Some required environment variables are missing")

# Creating the connection string for SQLAlchemy
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Creating an SQLAlchemy engine
engine = create_engine(connection_string)

def load_data(csv_file, table_name):
    try:
        data = pd.read_csv(csv_file)
        data.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        print(f"Data from {csv_file} has been loaded into the {table_name} table.")
    except Exception as e:
        print(f"Error loading data from {csv_file} into {table_name}: {e}")

# Load data from each CSV file into the corresponding table
load_data('recruiting_location.csv', 'recruiting_location')
load_data('terminated_location.csv', 'terminated_location')
load_data('completed_location.csv', 'completed_location')
load_data('active_location.csv', 'active_location')

# Disposing of the engine (optional, for good practice)
engine.dispose()